## LLM as judge evaluation

In **part 1** different models will perform the same 3 tasks as are in the ground truth model (and in the assistant). 
1. The first task is to check if a claim is checkable.
2. The next task is to retrieve all relevant information, and if available also additional context from the original source.
3. Finally a confirmation prompt will try to decide is the user confirms and wants to continue or not.

In **part 2**
Two larger different models will score the generated output of each model and task against the ground truth (the manually verified output). 
All tasks will be scored on a binary scale
1. The first task will be scored on *Reasoning*, with 0="reasoning not that clear", 1="clear reasoning"
2. The second task will be scored on:
    - *Completeness*, with 0="missing facts", 1="complete"
    - *Hallucination*, with 0="contains hallucinated facts", 1="no hallucinations found"
3. The last task will be scored on *Intend* (did the user want to proceed or not), with 0="confirmed differs", 1="confirmed is the same"

### Part 1 Generating output

In [177]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from tavily import TavilyClient

# Load alle the API keys
load_dotenv(dotenv_path="../.env", override=True)

# Initialize Tavily client 
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY", ""))

# Models on Groq, to evaluate
llama8 = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.1)
GPTOSS20 = ChatGroq(model_name="openai/gpt-oss-20b", model_kwargs={"tool_choice": "none"}, temperature=0.1)
qwen3_32 = ChatGroq(model_name="qwen/qwen3-32b", temperature=0.1)
llama70 = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.1)
GPTOSS120 = ChatGroq(model_name="openai/gpt-oss-120b", model_kwargs={"tool_choice": "none"}, temperature=0.1)

In [178]:
from typing import Literal,List
from pydantic import BaseModel, Field

class CheckResult(BaseModel):
    checkable: Literal["POTENTIALLY CHECKABLE", "UNCHECKABLE"]
    explanation: str = Field("")

class RetrieveInfoResult(BaseModel):
    claim_source: str = Field("unknown")
    primary_source: bool = Field(False)
    source_description: str = Field("")
    subject: str = Field("unclear")
    quantitative: str = Field("") 
    precision: str = Field("")
    based_on: str = Field("")
    alerts: str = Field("",description="A comma-separated list of flags or 'None' if no issues found")
    geography: str = Field("unclear")
    time_period: str = Field("unclear")

class ConfirmResult(BaseModel):
    confirmed: bool = Field(False)

In [179]:
from langchain_core.prompts import ChatPromptTemplate

checkable_check_prompt = """
### Role
Neutral Fact-Checking Analyst.

### Inputs
Claim: {claim}

### Task
Classify the claim and determine if it can be fact-checked.

### Classification Logic
- **UNCHECKABLE**: Opinion, value judgment, or prediction.
- **POTENTIALLY CHECKABLE**: Factual claims about the past or present.

### Output (JSON)
{{
  "checkable": "POTENTIALLY CHECKABLE",
  "explanation": "Brief justification for the classification."
}}
""".strip()

c_prompt = ChatPromptTemplate.from_template(checkable_check_prompt)

retrieve_info_prompt = """
### Role
Neutral Fact-Checking Analyst. Focus on objective evaluation.

### Context
- Claim: {claim}
- Year: {year}

### Additional context the user provided
"{additional_context}"

### Task 1: Source & Intent Extraction
1. **claim_source**: Identify the person or organization who originated the claim.
2. **primary_source**: Set to true ONLY if the evidence confirms this is the original/foundational origin.
3. **source_description**: Describe the medium (e.g., "Official PDF", "Social Media Post").

### Task 2: Factual Dimension Analysis
1. **Subject**: Identify the core entity or event.
2. **Quantitative/Qualitative**: Explain if it is measurable data or a description.
3. **Precision**: Categorize as Precise, Vague, or Absolute (100%), and provide specific numbers, or names from the evidence.
4. **Based On**: Identify the likely methodology (e.g., Official stats, Survey, research). Provide a brief explanation.
5. **Geography**: Identify the geographic scope of the claim.
6. **Time Period**: The time frame is normally {year}, unless otherwise stated in the claim or additional context.

### Task 3: Guidance & Risk
1. **Alerts**: Flag "missing geography", "qualitative claim", "missing methodological details", "missing source". 
  If the claim is quantitative, but no numbers are provide or textual precise indications (e.g., "All", "None"), all), *Flag in Alerts*: "vague quantitative claim".
  Do not flag if the info is present.
2. Include specific details (dates, numbers, names) from the additional context if available:
"{additional_context}"

### Output Format (JSON)
{{
  "claim_source": "a Person or an Organisation" or "unknown"
  "primary_source": true/false
  "source_description": "medium description"
  "subject": "subject text" or "unclear"
  "quantitative": "quantitative/qualitative and a  short explanation"
  "precision": "precise/vague/absolute and some specifics"
  "based_on": "methodology and short explanation" or "unclear"
  "alerts": "string containing all alerts separated by commas",
  "geography": "Geography",
  "time_period": "Year or more precise timeframe"
}}
""".strip()

r_prompt = ChatPromptTemplate.from_template(retrieve_info_prompt)

# Prompt to confirm the extracted information with the user
intent_prompt = """
### Role
Linguistic Analyst specializing in intent detection.

### Context
- User's Response: "{user_answer}"

### Task
Analyze the "User's Response" to determine if the assistant has permission to proceed to the next stage of the process ("Green Light").

### Decision Rules
**Set "confirmed": true IF:**
1. **Explicit Command:** The user uses navigational keywords (e.g., "Next," "Continue," "Proceed," "Move on," "Go ahead").
2. **Affirmation:** The user agrees with a previous summary (e.g., "Yes," "Correct," "Exactly," "That's right").
3. **Closure:** The user indicates they have no further details to provide (e.g., "I don't know," "That's all I have," "No more info").

**Set "confirmed": false IF:**
1. **New Info Only:** The user provides additional facts or context but does NOT include a command to move on.
2. **Correction:** The user is correcting a previous mistake.
3. **Uncertainty:** The user asks a follow-up question or expresses confusion.

### Important Rules
- Focus on the **intent to progress**.
- If the response is ambiguous, default to `false`.

### Output (JSON)
{{
  "confirmed": true
}}
""".strip()

i_prompt = ChatPromptTemplate.from_template(intent_prompt)


In [180]:
# Function to add checkable columns to DataFrame
def add_model_columns(
        df: pd.DataFrame, 
        retrieval_chain, 
        check_chain,
        intent_chain,
        model_name: str, 
        claim_col: str = "claim", 
        context_col: str = "translated", 
        year_col: int = "year",
        user_answer_col: str = "user_answer"
    ) -> pd.DataFrame:
    
    # Copy the dataframe
    out = df.copy()

    # For each row in the dataset call the llm
    def _run_row(row):
        claim = row[claim_col]
        translated = row[context_col]
        year = row[year_col]
        user_answer= row[user_answer_col]
        additional_context = translated if isinstance(translated, str) and translated.strip() else ""

        check= check_chain.invoke({
            "claim": claim,
        })

        retrieval= retrieval_chain.invoke({
            "claim": claim,
            "year":year,
            "additional_context": additional_context
        })

        intent= intent_chain.invoke({
            "user_answer": user_answer,
        })

        # Build the human-readable summary per row
        details_text = (
            f"- claim_source: {retrieval.claim_source}\n"
            f"- primary_source: {retrieval.primary_source}\n"
            f"- source_description: {retrieval.source_description}\n"
            f"- subject: {retrieval.subject}\n"
            f"- quantitative: {retrieval.quantitative or 'not clearly specified'}\n"
            f"- precision: {retrieval.precision}\n"
            f"- based_on: {retrieval.based_on}\n"
            f"- geography: {retrieval.geography}\n"
            f"- time_period: {retrieval.time_period}\n"
        )
        

        # Return a Series so we can easily join it back to the dataframe
        return pd.Series({
            f"checkable_{model_name}": check.checkable,
            f"explanation_{model_name}": check.explanation,
            f"details_{model_name}": details_text,
            f"alerts_{model_name}": retrieval.alerts,
            f"confirmed_{model_name}": intent.confirmed,
        })

    # Apply the function and join the new columns to the original dataframe
    results = out.apply(_run_row, axis=1)
    out = pd.concat([out, results], axis=1)

    return out

In [ ]:
import pandas as pd

# Build the langchain chain
def build_check_chain(llm):
    structured_llm = llm.with_structured_output(CheckResult, method="json_mode")
    return c_prompt | structured_llm

# Build the langchain chain
def build_retrieval_chain(llm):
    structured_llm = llm.with_structured_output(RetrieveInfoResult, method="json_mode")
    return r_prompt | structured_llm

# Build the langchain chain
def build_intent_chain(llm):
    structured_llm = llm.with_structured_output(ConfirmResult, method="json_mode")
    return i_prompt | structured_llm

# Build chains for Llama8
llama8_retrieval = build_retrieval_chain(llama8)
llama8_check = build_check_chain(llama8)
llama8_intent = build_intent_chain(llama8)

# Build chains for GPT OSS-20B
gpt20_retrieval = build_retrieval_chain(GPTOSS20)
gpt20_check = build_check_chain(GPTOSS20)
gpt20_intent = build_intent_chain(GPTOSS20)

# Build chains for GPT OSS-120B
gpt120_retrieval = build_retrieval_chain(GPTOSS120)
gpt120_check = build_check_chain(GPTOSS120)
gpt120_intent = build_intent_chain(GPTOSS120)

# Build chains for Qwen2-32B
qwen_retrieval = build_retrieval_chain(qwen3_32)
qwen_check = build_check_chain(qwen3_32)
qwen_intent = build_intent_chain(qwen3_32)

# Build chains for Llama70
llama70_retrieval = build_retrieval_chain(llama70)
llama70_check = build_check_chain(llama70)
llama70_intent = build_intent_chain(llama70)

# Map the model nicknames to the chains
models_to_run = {
    "llama8": {"retrieval": llama8_retrieval, "check": llama8_check, "intent": llama8_intent},
    "llama70": {"retrieval": llama70_retrieval, "check": llama70_check, "intent": llama70_intent},
    "gptoss20": {"retrieval": gpt20_retrieval, "check": gpt20_check, "intent": gpt20_intent},
    "gptoss120": {"retrieval": gpt120_retrieval, "check": gpt120_check, "intent": gpt120_intent},
    "qwen3_32": {"retrieval": qwen_retrieval, "check": qwen_check, "intent": qwen_intent}
}

# Load your ground dataset
df_models = pd.read_csv("validated_reference_data.csv", encoding="utf-8")

# Loop through the dictionary
for name, chains in models_to_run.items():
    print(f"Running evaluation for: {name}...")
    df_models = add_model_columns(
        df_models,
        retrieval_chain=chains["retrieval"],
        check_chain=chains["check"],
        intent_chain=chains["intent"],
        model_name=name
    )

# 4. Save the final combined results
df_models.to_csv("eval_models_output.csv", index=False)

Running evaluation for: llama8...
Running evaluation for: gptoss20...


### Part 2 Judging the output

In [156]:
from typing import Literal,List
from pydantic import BaseModel, Field

class Task1Result(BaseModel):
    reasoning: bool

class Task2Result(BaseModel):
    hallucination: bool
    completeness: float

In [157]:
from langchain_core.prompts import ChatPromptTemplate

score_task1_prompt = """
### Role
You are an expert Logical Analyst acting as an LLM Judge. 
Your goal is to perform a semantic and logical comparison between a validated reference data and a generated candidate output.

### Inputs
- Claim: {claim}

### Reference
- **Explanation (Reference):** 
<RExplanation>{r_explanation}</RExplanation>

- **Rating (Reference):** {r_rating}

### Candidate 
- **Explanation (Candidate):** 
<CExplanation>{c_explanation}</CExplanation>

- **Rating (Candidate):** {c_rating}

### Evaluation Objective
Determine if the **Candidate Explanation** aligns with the **Reference Explanation** or provides a logically sound alternative based solely on the provided Claim.

### Judgment Principles
- **Logical Path**: Prioritize the underlying premises and causal links over tone, vocabulary, or verbosity.
- **Rating Consistency**: The Candidate must reach the same conclusion or "rating" as the Reference.
- **Permissible Variance**: Mark as TRUE if the Candidate uses a different but valid logical approach to reach the same conclusion, provided it does not contradict known facts.

### Scoring Rubric
**Set "reasoning": true IF:**
- The core logical flow matches the Reference reasoning.
- OR the reasoning differs but is factually correct, logically consistent, and reaches the same final rating.
- All stated facts are accurate and no critical steps in the chain are missing.

**Set "reasoning": false IF:**
- The candidate introduces factual hallucinations (data or entities not in the claim/reference).
- The explanation contains logical fallacies or misses a critical constraint.
- The candidate suggests/implies a different fact-checking outcome than the Reference.

### Output Format
Return a JSON object with the following keys:
### Output (JSON)
{{
  "reasoning": true/false,
}}
""".strip()

t1_prompt = ChatPromptTemplate.from_template(score_task1_prompt)

score_task2_prompt = """
### Role
You are a Precision Auditor. Your task is to evaluate a generated response based on a set of provided source materials.

### Inputs
- Claim: {claim}
- Year: {year}

### Additional context the user provided
"{additional_context}"

### Reference
- **Details(Reference):** 
<RDetails>{r_details}</RDetails>

- **Alerts (Reference):** 
<RDetails>{r_alerts}</RDetails>

### Candidate 
- **Details (Candidate):**
<CDetails>{c_details}</CDetails>

- **Alerts (Candidate):**
<CDetails>{c_alerts}</CDetails>

### Evaluation Objective 1: Completeness
Assess if the **Candidate Data** includes the critical information, numbers, and alerts found in the **Reference Data**.

**Judgment Principles**:
  - Focus on essential facts: names, specific numbers, and methodological approach.
  - Check for Alignment: Ensure the most critical "Alerts" or "Flags" from the Reference are present.
  - **Leniency**: Do not penalize for minor differences in "Quantitative" vs "Qualitative" labeling, as these are often subjective.

### Evaluation Objective 2: Hallucination
Determine if the **Candidate Data** introduces information that is not supported by the provided sources.

**Judgment Principles**:
  - **No Hallucination (1)**: Every piece of evidence in the details or alerts exists strictly within the **Claim** or the **Additional Context**.
  - **Hallucination (0)**: The response includes "made up" facts, figures, or external knowledge not explicitly found in the provided context.

### Scoring Rubric

#### **1. Completeness Score (0.0 to 1.0)**
- **1.0**: All critical facts, numbers, and alerts from the Reference are present.
- **0.5**: Some key details are missing, but the core essence remains.
- **0.0**: Critical names, numbers, or alerts are entirely absent.

#### **2. Hallucination Free (Boolean)**
- **true**: Every claim in the Candidate is grounded in the Claim/Context.
- **false**: The Candidate introduces information from outside the provided text.

### Output (JSON)
{{
  "completeness": 0.0 to 1.0
  "hallucination": true/false
}}.strip()
"""
t2_prompt = ChatPromptTemplate.from_template(score_task2_prompt)


In [158]:
from typing import List

def run_multi_model_evaluation(
    df: pd.DataFrame, 
    task1_chain, 
    task2_chain, 
    model_suffixes: List[str],
) -> pd.DataFrame:
    """
    Runs evaluation for multiple models and returns a NEW dataframe
    with the results.
    """
    # Copy of the dataframe
    out_df = df.copy()

    for suffix in model_suffixes:
        print(f"Evaluating model: {suffix}...")
        
        # Define dynamic column names based on the suffix
        c_expl_col = f"explanation_{suffix}"
        c_rating_col = f"checkable_{suffix}"
        c_details_col = f"details_{suffix}"
        c_alerts_col = f"alerts_{suffix}"
        c_confirmed_col = f"confirmed_{suffix}"
        
        # Reference columns (assuming these are constant/static)
        r_expl_col = "explanation"
        r_rating_col = "checkable"
        r_details_col = "details_text"
        r_alerts_col = "alerts"
        r_confirmed_col = "confirmed"

        # primary source data
        claim_col = "claim"
        translated = "translated"
        year_col = "year"

        def _evaluate_row(row):

            # Get the additional context from the translated column, if it is not empty
            additional_context = row["translated"] if "translated" in row and isinstance(row["translated"], str) and row["translated"].strip() else ""

            # Reasoning Comparison
            t1_output = task1_chain.invoke({
                "r_explanation": row[r_expl_col],
                "c_explanation": row[c_expl_col],
                "r_rating": row[r_rating_col],
                "c_rating": row[c_rating_col],
                "claim": row[claim_col],
            })

            # Completeness & Hallucination
            t2_output = task2_chain.invoke({
                "r_details": row[r_details_col],
                "c_details": row[c_details_col],
                "r_alerts": row[r_alerts_col],
                "c_alerts": row[c_alerts_col],
                "claim": row[claim_col],
                "year":row[year_col],
                "additional_context": additional_context,
            })

            # Check confirmed columns
            intent = row[r_confirmed_col] == row[c_confirmed_col]

            return (
                t1_output.reasoning,
                t2_output.completeness,
                t2_output.hallucination,
                intent,
            )

        # Apply the evaluation
        temp_results = out_df.apply(_evaluate_row, axis=1)

        # Unpack results into new columns in our results_df
        out_df[f"reason_{suffix}"] = temp_results.apply(lambda t: t[0])
        out_df[f"complete_{suffix}"] = temp_results.apply(lambda t: t[1])
        out_df[f"halluci_{suffix}"] = temp_results.apply(lambda t: t[2])
        out_df[f"intent_{suffix}"] = temp_results.apply(lambda t: t[3])

    return out_df

Finally we will run the evaluation calculating all the scores

In [159]:
# these models are being evaluated
models_to_test = ["llama8", "gptoss20"]

# Build the langchain chain
def build_task1_chain(llm):
    structured_llm = llm.with_structured_output(Task1Result, method="json_mode")
    return t1_prompt | structured_llm

# Build the langchain chain
def build_task2_chain(llm):
    structured_llm = llm.with_structured_output(Task2Result, method="json_mode")
    return t2_prompt | structured_llm

# Build chains for GPT
task1_chain = build_task1_chain(GPTOSS120)
task2_chain = build_task2_chain(GPTOSS120)

# Load your ground dataset
original_df = pd.read_csv("eval_models_output.csv", encoding="utf-8")

scores_df = run_multi_model_evaluation(original_df, task1_chain, task2_chain, models_to_test)

scores_df.to_csv("eval_gptoss_output.csv", index=False)

Evaluating model: llama8...
Evaluating model: gptoss20...


In [162]:
#scores_df[["reason_llama8","complete_llama8","halluci_llama8","intent_llama8","reason_gptoss20","complete_gptoss20","halluci_gptoss20","intent_gptoss20","reason_qwen3_32","complete_qwen3_32","halluci_qwen3_32","intent_qwen3_32"]].sum()
scores_df[["reason_llama8","complete_llama8","halluci_llama8","intent_llama8","reason_gptoss20","complete_gptoss20","halluci_gptoss20","intent_gptoss20"]].mean()

reason_llama8        0.865772
complete_llama8      0.661074
halluci_llama8       0.523490
intent_llama8        0.664430
reason_gptoss20      0.825503
complete_gptoss20    0.597315
halluci_gptoss20     0.489933
intent_gptoss20      1.000000
dtype: float64

Next to the LLM as judge approach, Bertscore was also used to measure the simularity between the reference an candidate texts

In [173]:
from bert_score import BERTScorer
scorer = BERTScorer(model_type="distilroberta-base",lang="en",rescale_with_baseline=True,device="cuda")

# Copy of dataset
metrics_df = scores_df.copy()

# Models to evaluate
models = ["llama8", "gptoss20"]

# Reference texts
r_explanation = metrics_df["explanation"].fillna("").astype(str).tolist()
r_details = metrics_df["details_text"].fillna("").astype(str).tolist()

for model in models:
    print(f"Computing BERTScore for {model}...")

    # Candidate texts
    c_explanation = metrics_df[f"explanation_{model}"].fillna("").astype(str).tolist()
    c_details = metrics_df[f"details_{model}"].fillna("").astype(str).tolist()

    # Explanation scores
    P_e, R_e, F1_e = scorer.score(c_explanation, r_explanation)

    # Details scores
    P_d, R_d, F1_d = scorer.score(c_details, r_details)

    # Store results
    metrics_df[f"P_explanation_{model}"] = P_e.cpu().numpy()
    metrics_df[f"R_explanation_{model}"] = R_e.cpu().numpy()
    metrics_df[f"F1_explanation_{model}"] = F1_e.cpu().numpy()

    metrics_df[f"P_details_{model}"] = P_d.cpu().numpy()
    metrics_df[f"R_details_{model}"] = R_d.cpu().numpy()
    metrics_df[f"F1_details_{model}"] = F1_d.cpu().numpy()

Computing BERTScore for llama8...
Computing BERTScore for gptoss20...


And create a better overview

In [175]:
rows = []

for model in models:
    rows.append({
        "model": model,
        "reasoning": scores_df[f"reason_{model}"].mean(),
        "hallucination": scores_df[f"halluci_{model}"].mean(),
        "completeness": scores_df[f"complete_{model}"].mean(),
        "intent": scores_df[f"intent_{model}"].mean(),

        "P_e": scores_df[f"P_explanation_{model}"].mean(),
        "R_e": scores_df[f"R_explanation_{model}"].mean(),
        "F1_e": scores_df[f"F1_explanation_{model}"].mean(),

        "P_d": scores_df[f"P_details_{model}"].mean(),
        "R_d": scores_df[f"R_details_{model}"].mean(),
        "F1_d": scores_df[f"F1_details_{model}"].mean(),
    })

summary_df = pd.DataFrame(rows).set_index("model")
summary_df

,reasoning,hallucination,completeness,intent,P_e,R_e,F1_e,P_d,R_d,F1_d
model,,,,,,,,,,
llama8,0.865772,0.523490,0.661074,0.66443,0.570290,0.576105,0.573177,0.788007,0.650249,0.718141
gptoss20,0.825503,0.489933,0.597315,1.00000,0.678966,0.645880,0.662246,0.788700,0.673410,0.730230
